# Test RNN sur le déjeuner diner
prise en compte de l'identité utilisateur pour les repas
tests d'optimisation des hyperparamètres sur le modèle contrôle où les user embeddings ne sont pas intégrés

In [1]:
# !pip install torchtext
from packaging import version
from pathlib import Path
from itertools import chain
from typing import Iterable, List, Tuple

import shutil
import torchtext
assert version.Version(torchtext.__version__) >= version.Version("0.9.0")
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab, FastText
from collections import Counter
from tqdm.autonotebook import tqdm
from torch.nn.utils.rnn import pad_sequence
import sentencepiece as spm
from IPython.display import display, HTML
import os


cachepath = os.path.expanduser('~/.local/data')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# outils avancés de gestion des chemins
BASEPATH = Path("xp/generation")
TB_PATH =  BASEPATH / "logs"
TB_PATH.mkdir(parents=True, exist_ok=True)

# usage externe de tensorboard: (1) lancer la commande dans une console; (2) copier-coller l'URL dans un navigateur
display(HTML("<h2>Informations</h2><div>Pour visualiser les logs, tapez la commande : </div>"))
print(f"tensorboard --logdir {Path(TB_PATH).absolute()}")

C:\Users\jacqueno\AppData\Local\Temp\ipykernel_48360\2790905849.py:20: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


tensorboard --logdir C:\Users\jacqueno\Documents\IODAA\EXERSYS\final_to_git\Modele_RNN\Dej_Din\xp\generation\logs


In [2]:
# !pip install sentencepiece
# !pip install datasets
# !pip install fsspec==2021.8.1
# !pip uninstall huggingface-hub

# A. Chargement des données

In [4]:
#ADAPTATION EXERSYS
import pickle
with open("REP_vraisalimfreqliste.pkl", "rb") as fichier:
    REP_vraisalimfreqliste = pickle.load(fichier)

In [5]:
#ADAPTATION EXERSYS
print(REP_vraisalimfreqliste[:20])

[['28301', '9811', '7001', '18008', '19611', '13030', '16632', '81007'], ['20047', '11110', '22501', '39221', '18008', '7001', '81007'], ['25163', '9811', '19640', '18008', '7001', '13037'], ['8400', '25103', '9104', '7001', '19640', '18008', '11110'], ['36005', '20071', '7001', '18008', '19611', '13014', '81007', '81007'], ['28925', '20071', '11110', '7001', '18008', '39221', '81007'], ['36306', '20101', '4003', '17440', '28504', '11081', '7001', '19640', '18008', '81007', '81007'], ['28900', '7001', '19624', '18008'], ['6210', '20030', '7001', '23525', '18008'], ['26034', '25606', '11110', '7001', '23525', '18008'], ['25902', '28916', '76057', '19640', '13039', '7100'], ['9811', '13039', '19700', '76022', '11114'], ['20009', '1007', '25902', '9811', '11114', '12114', '18009', '81061'], ['25902', '25103', '9104', '12114', '18008', '7001', '81029', '81009'], ['25906', '8400', '25606', '11110', '19700', '7001', '18008', '13037'], ['9811', '11107', '81022'], ['76061', '13034', '20051', '

In [7]:
#ADAPTATION EXERSYS
#Créer des jeux de training, validation, set

import random
import pickle
data=REP_vraisalimfreqliste

# # Define the proportions for train, validation, and test datasets (as percentages)
# train_ratio = 0.7
# validation_ratio = 0.15
# test_ratio = 0.15

# # Calculate the number of samples for each dataset
# num_samples = len(data)
# num_train = int(num_samples * train_ratio)
# num_validation = int(num_samples * validation_ratio)
# num_test = num_samples - num_train - num_validation

# # Shuffle the data randomly
# random.shuffle(data)

# # Split the data into three datasets: train, validation, and test
# train_data = data[:num_train]
# validation_data = data[num_train:num_train + num_validation]
# test_data = data[num_train + num_validation:]


# # Save the train_data using pickle
# with open('train_data.pkl', 'wb') as f:
#     pickle.dump(train_data, f)

# # Save the validation_data  using pickle
# with open('validation_data.pkl', 'wb') as g:
#     pickle.dump(validation_data, g)

# # Save the test_data using pickle
# with open('test_data.pkl', 'wb') as h:
#     pickle.dump(test_data, h)

# Load the train_data from the saved file
with open('train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)

# Load the validation_data from the saved file
with open('validation_data.pkl', 'rb') as g:
    validation_data = pickle.load(g)

# Load the test_data from the saved file
with open('test_data.pkl', 'rb') as h:
    test_data = pickle.load(h)

# Now, 'train_data', 'validation_data', and 'test_data' contain the respective datasets
print("Train Data:", train_data[:20])
print("Validation Data:", validation_data[:20])
print("Test Data:", test_data[:20])

# Convertir les sous-listes en phrases (chaînes de caractères)
train_data_sentences = [' '.join(sublist) for sublist in train_data]
validation_data_sentences = [' '.join(sublist) for sublist in validation_data]
test_data_sentences = [' '.join(sublist) for sublist in test_data]
print(train_data_sentences[:2])

all_sentences = []
all_sentences.extend(train_data_sentences)
all_sentences.extend(validation_data_sentences)
all_sentences.extend(test_data_sentences)
print(len(all_sentences))

Train Data: [['5206', '20047', '12115', '13034', '7001', '11017', '81063'], ['76020'], ['25405', '36005', '4018', '23490', '18066', '5206', '18004'], ['25410', '28916', '18066', '12100', '22010', '13008'], ['9811', '12114', '18066', '18017'], ['6255', '20030', '5210', '18008'], ['20009', '6999', '9822', '13005', '76056', '81024', '81008', '81063'], ['9811', '28504', '19402', '13024', '18066', '81009'], ['9811', '12020', '13014', '31016', '18008'], ['13037', '19760', '76020'], ['30742', '4010', '90068', '23477', '7001', '18066', '18004'], ['9811', '6255', '11107', '20035', '5001', '5205', '7110', '81007', '81007'], ['4003', '20030', '17270', '26025', '5205', '90068', '13039', '7001', '18004', '81052', '81022', '81007', '81006', '81007', '81006'], ['25002', '9104', '7001', '19628', '81024', '81009'], ['18066', '6200', '4018', '90068', '23585', '7001'], ['26015', '17440', '25031', '13044', '5205', '18008', '7001', '18004', '31016', '81022'], ['25902', '19606', '18008', '81052'], ['1006', 

In [8]:
# Aplatir la liste de listes
aplatie = [item for sublist in REP_vraisalimfreqliste for item in sublist]
# Obtenir les valeurs uniques
features = list(set(aplatie))

print(len(features))
print(features)

335
['30155', '81022', '13005', '20015', '19402', '81063', '28900', '28916', '6210', '13043', '6520', '25512', '11158', '20066', '92051', '6255', '16402', '6110', '7012', '13030', '25003', '13008', '81009', '17440', '11110', '11008', '19800', '11004', '4019', '23684', '90110', '19510', '90031', '90057', '22014', '20101', '20002', '19602', '13033', '90067', '25435', '4027', '20062', '11000', '20035', '39511', '20045', '36018', '25906', '11054', '19606', '28202', '11014', '26140', '81006', '19628', '11107', '25056', '20037', '20051', '11018', '20071', '25088', '12100', '25945', '12114', '20030', '6200', '26025', '1003', '30300', '20076', '5200', '23490', '8305', '13021', '9104', '12500', '17130', '8800', '21501', '26039', '13009', '20099', '23000', '17001', '13004', '13039', '20068', '7255', '11017', '11108', '8704', '5001', '1018', '39206', '28101', '81037', '25163', '13134', '81024', '13034', '25405', '81000', '25902', '28802', '25018', '25947', '19590', '25524', '11161', '2007', '2352

# B. Preprocessing

In [9]:
#ADAPTATION EXERSYS
class Preprocessor:
    def process_batch(self, batch: List[str]):
        return pad_sequence([self.string2idx(text) for text in batch])

class SentencePiecePreprocessor(Preprocessor):
    """Tokenizer Sentence Piece"""
    def embeddings(self, dimension: int):
        return nn.Embedding(self.vocab_size, dimension, padding_idx=self.pad_index)

    @property
    def vocab_size(self):
        return self.spm_tokenizer.vocab_size()

    def __init__(self, name: str, sentences: Iterable[str], vocab_size=len(features)+4, force=False):
        modelpath = BASEPATH / f"{name}-{vocab_size}.model"
        if not modelpath.exists() or force:
            print(f"Entraînement de SPM, sortie {BASEPATH}/{name}-{vocab_size}", flush=True)
            spm.SentencePieceTrainer.train(
                sentence_iterator=iter(sentences),
                model_prefix=f"{BASEPATH}/{name}-{vocab_size}",
                vocab_size=vocab_size,
                model_type="word",
                pad_id=0,
                unk_id=1,
                bos_id=2,
                eos_id=3
            )

        self.spm_tokenizer = spm.SentencePieceProcessor()
        self.spm_tokenizer.load(str(modelpath))

        self.pad_index = self.spm_tokenizer.pad_id()
        self.eos_index = self.spm_tokenizer.eos_id()
        self.bos_index = self.spm_tokenizer.bos_id()
        self.oov_index = None

        self.spm_tokenizer.SetEncodeExtraOptions("bos:eos")

    def string2idx(self, s: str) -> torch.Tensor:
        return torch.tensor(self.spm_tokenizer.EncodeAsIds(s))

    def tokenizer(self, x):
        """Segmentation du texte en sous-mots"""
        return self.spm_tokenizer.encode_as_pieces(x)

    def id2token(self, ix):
        return self.spm_tokenizer.IdToPiece(ix)

    def decode(self, ids: List[int]):
        return self.spm_tokenizer.Decode(ids)


preprocessor = SentencePiecePreprocessor("spm", all_sentences, force=False)
print(f"Vocabulary size: {preprocessor.vocab_size}")

Entraînement de SPM, sortie xp\generation/spm-339
Vocabulary size: 339


In [10]:
#ADAPTATION EXERSYS
# exemple de décomposition
preprocessor.tokenizer(['25902 25019 13039 5210 13005 18066'])

[['<s>', '▁25902', '▁25019', '▁13039', '▁5210', '▁13005', '▁18066', '</s>']]

In [11]:
#ADAPTATION EXERSYS
out = preprocessor.process_batch(['25902 25019 13039 5210 13005 18066']) # => tenseur d'indices présenté en colonne
print(out)
print(out.shape)

tensor([[  2],
        [ 17],
        [113],
        [ 11],
        [ 51],
        [ 32],
        [  5],
        [  3]])
torch.Size([8, 1])


In [12]:
#ADAPTATION EXERSYS
# transformation inverse (index => token)
# attention (toujours aux dimensions) : si on oublie le squeeze => list de token

print(preprocessor.decode(out.tolist()))
print(preprocessor.decode(out.squeeze().tolist()))

['', '25902', '25019', '13039', '5210', '13005', '18066', '']
25902 25019 13039 5210 13005 18066


In [ ]:
#ADAPTATION EXERSYS
# reflexion sur le padding
x_exemple  = preprocessor.process_batch(['25902 25019 13039 5210 13005 18066', '28925 20066 20047 11110 12115 9811 9811 9811'])

print("Dimension de x :", x_exemple .size()) # T x batch_size x dim_input
print(x_exemple )

Dimension de x : torch.Size([10, 2])
tensor([[  2,   2],
        [ 17, 180],
        [113, 158],
        [ 11,  14],
        [ 51,  18],
        [ 32, 138],
        [  5,   7],
        [  3,   7],
        [  0,   7],
        [  0,   3]])


In [ ]:
#ADAPTATION EXERSYS
# récupération automatique d'une lookup table (tableau d'embeddings):
embeddings = preprocessor.embeddings(50) # taille vaocabulaire x dim = 50

print("taille de la table:", embeddings.num_embeddings, "x", embeddings.embedding_dim)

taille de la table: 339 x 50


In [ ]:
#ADAPTATION EXERSYS
# verification sur la nature de la représentation du mot de padding

x = embeddings(preprocessor.process_batch(['25902 25019 13039 5210 13005 18066', '28925 20066 20047 11110 12115 9811 9811 9811']))
print("Dimension de x :", x.size()) # T x batch_size x dim_input
print("mot :", x[9,0,:].data) # 9e mot du doc 0 (toutes les dimensions de la représentation)

Dimension de x : torch.Size([10, 2, 50])
mot : tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [ ]:
# avant entrainement du modèle
z = embeddings(preprocessor.process_batch(['25902']))
print("Dimension de z :", z.size())
print(z)

Dimension de z : torch.Size([3, 1, 50])
tensor([[[ 1.3749, -0.8125,  1.4249,  0.4585,  0.1057, -1.0082,  1.1179,
           0.8327,  0.1201,  0.8923,  0.5946,  0.9276,  0.4544, -0.6387,
           1.3188,  1.6178,  0.8457,  0.1210, -0.0503,  0.0922, -0.4320,
          -0.6997, -0.4772, -0.6807, -0.9946,  0.7547, -0.5677,  0.4694,
           0.1605, -0.2666,  1.0335,  0.9379,  0.0318, -1.6428, -0.7637,
           1.0541, -2.5984, -0.7315,  1.2499,  0.4212, -1.4539,  0.7911,
          -1.2214, -0.5598, -1.4925, -0.4376,  0.5514,  0.8215,  0.9148,
           0.1699]],

        [[ 2.5989, -0.8572,  0.0972, -0.8542, -0.9607,  0.0778, -0.6295,
           1.1954,  1.6021,  0.7728,  0.2431,  0.3142, -1.3650, -0.5571,
          -0.0660, -0.7378, -1.4272, -0.2971,  2.4327, -0.2205,  0.7031,
           0.3904, -0.4577,  0.1796, -1.0801,  1.2974,  1.2830,  0.2528,
           0.8811, -1.4513,  0.6019,  1.1398, -0.5457, -0.8302,  3.2223,
           0.4526,  0.2707,  0.7356, -0.4615, -0.1716, -0.0878

In [13]:
#Vérifier la bonne tokenisation de tous les aliments
rand_vec=0
for token_id in range(preprocessor.vocab_size):
    token = preprocessor.id2token(token_id)
    print(token)
    if token.startswith("▁"):
        if not str(token[1:]) in features:
            rand_vec+=1
            print("token not in model:", token)


<pad>
<unk>
<s>
</s>
▁7001
▁18066
▁18008
▁9811
▁18004
▁25604
▁81024
▁13039
▁81008
▁28916
▁20047
▁81009
▁31016
▁25902
▁11110
▁16400
▁17270
▁9104
▁90068
▁81022
▁6255
▁12114
▁81052
▁11018
▁5205
▁19600
▁81007
▁5203
▁13005
▁13024
▁12999
▁36005
▁4003
▁6200
▁18018
▁81025
▁7100
▁76020
▁7012
▁19628
▁22010
▁11017
▁20009
▁20062
▁4027
▁19624
▁22500
▁5210
▁4010
▁25903
▁13037
▁20030
▁81011
▁4005
▁81010
▁81006
▁25516
▁28101
▁17440
▁13034
▁11108
▁18044
▁11107
▁20026
▁81061
▁13026
▁28301
▁81029
▁11054
▁36306
▁20031
▁76056
▁19402
▁8211
▁5206
▁76016
▁30300
▁20035
▁20210
▁92044
▁4019
▁11013
▁20008
▁26030
▁4018
▁26039
▁26140
▁19800
▁92024
▁13044
▁28504
▁11015
▁25018
▁76057
▁18071
▁25405
▁18009
▁12500
▁28800
▁4006
▁20021
▁81042
▁19606
▁20003
▁17001
▁19640
▁19765
▁13043
▁20071
▁25019
▁36004
▁81038
▁12001
▁20011
▁20027
▁11109
▁6210
▁19501
▁22501
▁25127
▁11000
▁19510
▁30780
▁20099
▁23499
▁13014
▁25608
▁16402
▁5001
▁36018
▁11014
▁23490
▁25002
▁81012
▁12115
▁20017
▁20036
▁25056
▁17700
▁39200
▁13030
▁20045
▁81037

In [14]:
#detecter les erreurs de tokenisation par sentencepiece
import copy
features_miss=copy.deepcopy(features)
for token_id in range(preprocessor.vocab_size):
    token = preprocessor.id2token(token_id)
    if token.startswith("▁"):
        features_miss.remove(str(token[1:]))
print(features_miss)

[]


# C. RNN


In [18]:
import pickle

filename = "dic_userseqrepas.pkl"
with open(filename, "rb") as file:
    dic_userseqrepas = pickle.load(file)

print(dic_userseqrepas)

{110006: [['28301', '9811', '7001', '18008', '19611', '13030', '16632', '81007'], ['20047', '11110', '22501', '39221', '18008', '7001', '81007'], ['25163', '9811', '19640', '18008', '7001', '13037'], ['8400', '25103', '9104', '7001', '19640', '18008', '11110'], ['36005', '20071', '7001', '18008', '19611', '13014', '81007', '81007'], ['28925', '20071', '11110', '7001', '18008', '39221', '81007'], ['36306', '20101', '4003', '17440', '28504', '11081', '7001', '19640', '18008', '81007', '81007'], ['28900', '7001', '19624', '18008'], ['6210', '20030', '7001', '23525', '18008'], ['26034', '25606', '11110', '7001', '23525', '18008']], 110007: [['25902', '28916', '76057', '19640', '13039', '7100']], 110020: [['9811', '13039', '19700', '76022', '11114'], ['20009', '1007', '25902', '9811', '11114', '12114', '18009', '81061']], 110021: [['25902', '25103', '9104', '12114', '18008', '7001', '81029', '81009'], ['25906', '8400', '25606', '11110', '19700', '7001', '18008', '13037']], 110025: [['9811',

In [19]:
import copy
import numpy as np

# print(dic_userseqrepas)
dic_userseqrepas_min3=copy.deepcopy(dic_userseqrepas)
for cle, valeur in dic_userseqrepas.items():
        if len(valeur)<3:
#             print(cle, len(valeur))
            del dic_userseqrepas_min3[cle]

print(len(dic_userseqrepas))
print(len(dic_userseqrepas_min3))

dic_user_repas_train={}
dic_user_repas_valid={}
dic_user_repas_test={}
for cle, valeur in dic_userseqrepas_min3.items():
    r=0
    for repas in valeur:
        r+=1
        if r<len(valeur)-1:
            dic_user_repas_train.setdefault(cle, []).append(repas)
        elif r==len(valeur)-1:
            dic_user_repas_valid.setdefault(cle, []).append(repas)
        elif r==len(valeur):
            dic_user_repas_test.setdefault(cle, []).append(repas)
# dic_user_repas_train

train_data_old=[]
train_user_ids=[]
valid_data_old=[]
valid_user_ids=[]
test_data_old=[]
test_user_ids=[]

for cle, valeur in dic_user_repas_train.items():
    for repas in valeur:
        train_data_old.append(repas)
        train_user_ids.append(cle)
# print(len(train_data_old))
# print(len(train_user_ids))

for cle, valeur in dic_user_repas_valid.items():
    for repas in valeur:
        valid_data_old.append(repas)
        valid_user_ids.append(cle)
# print((valid_data_old))
# print((valid_user_ids))

for cle, valeur in dic_user_repas_test.items():
    for repas in valeur:
        test_data_old.append(repas)
        test_user_ids.append(cle)

print(len(np.unique(train_user_ids)))
print(len(np.unique(valid_user_ids)))
print(len(np.unique(test_user_ids)))


2385
1789
1789
1789
1789


In [20]:
#ADAPTATION EXERSYS
#adaptation avec user embedding : 2 façons de l'intégrer
# partie commune aux 2

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Créez un dictionnaire pour mapper les user_ids aux indices
user_id_to_index = {user_id: index for index, user_id in enumerate(np.unique(valid_user_ids))}
# print(user_id_to_index)


train_data=[' '.join(sublist) for sublist in train_data_old]
# print(train_data)
valid_data=[' '.join(sublist) for sublist in valid_data_old]
# print(valid_data)
test_data=[' '.join(sublist) for sublist in test_data_old]
# print(test_data)

train_batchsize = 100
valid_batchsize = len(valid_data)
test_batchsize = len(test_data)
print(valid_batchsize)
print(test_batchsize)

class CustomDataset(Dataset):
    def __init__(self, data, user_ids, user_id_to_index):
        self.data = data  # Les séquences de données
        self.user_ids = user_ids  # Les identifiants d'utilisateur
        self.user_id_to_index = user_id_to_index

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Retourner un tuple contenant la séquence et l'indice d'utilisateur
        user_id = self.user_ids[idx]
        user_index = self.user_id_to_index[user_id]
        return self.data[idx], user_index

train_dataset = CustomDataset(train_data, train_user_ids, user_id_to_index)
train_loader = DataLoader(train_dataset, train_batchsize, shuffle=True)

valid_dataset = CustomDataset(valid_data, valid_user_ids, user_id_to_index)
valid_loader = DataLoader(valid_dataset, valid_batchsize, shuffle=False)

test_dataset = CustomDataset(test_data, test_user_ids, user_id_to_index)
test_loader =  DataLoader(valid_dataset, test_batchsize, shuffle=False)

# for batch in train_loader:
#     sequences, user_indices = batch
#     print("Batch of Sequences:", sequences)
#     print("Batch of User Indices:", user_indices)



1789
1789


In [21]:
# charger dictionnaire des noms d'aliments
with open("dic_alim.pkl", "rb") as fichier:
    dic_alim = pickle.load(fichier)

In [23]:
# charger dictionnaire des catégories d'aliments associées aux aliments (les catégories csontruites par connaissance expert)
with open("dic_alimnonrare_cat_repas.pkl", "rb") as fichier:
    dic_alimnonrare_cat_repas = pickle.load(fichier)

In [24]:
# trouver clé à partir de valeur
def get_keys_by_value(dictionary, value):
    keys = []
    for key, val in dictionary.items():
        if value in val:
            keys.append(key)
    return keys

In [25]:
#retrouver une catégorie à partir d'un aliment
print(int(preprocessor.decode(17)))
get_keys_by_value(dic_alimnonrare_cat_repas, int(preprocessor.decode(17)))


25902


[('38',)]

In [ ]:
# CONTROLE : sans intégration du user embedding
#Optimisation des hyperparamètres sur le modèle controle sans intégration des users embeddings
# ATTENTION EXECUTE SUR GOOGLE COLAB !!!! 

class RNN(nn.Module):
    def __init__(self, embeddings,  hidden_dim):
        super().__init__()
        self.name = f"rnn-{embeddings.embedding_dim}-{hidden_dim}"
        self.embeddings = embeddings
        self.rnn = nn.RNN(embeddings.embedding_dim, hidden_dim, nonlinearity="tanh")
        self.fc = nn.Linear(hidden_dim, embeddings.num_embeddings)

    def forward(self, x, h_0=None):
        x = self.embeddings(x)
        output, hidden = self.rnn(x, h_0)
        return self.fc(output), hidden


hidden_dim_values = [10, 50, 100, 200]
embedding_dim_values=[10, 50, 100]
learning_rate_values = [0.001, 0.01]
epochs_values = [10, 20, 30, 50, 100, 200]

best_eval = None
best_params = None
perf={}

for hidden_dim in hidden_dim_values:
    for embed_d in embedding_dim_values:
        for learning_rate in learning_rate_values:
            for epochs in epochs_values:
                embeddings = preprocessor.embeddings(embed_d)
                model = RNN(embeddings, hidden_dim)
                optim = torch.optim.Adam(model.parameters(), lr=1e-3)
                model = model.to(device)
                loss = nn.CrossEntropyLoss(ignore_index=preprocessor.pad_index)  # classification avec une classe qui ne retourne pas d'erreur
                for epoch in tqdm(range(epochs)):
                    model.train()
                    for batch in train_loader:
                        sequences, user_index = batch
                        optim.zero_grad()
                        x = preprocessor.process_batch(sequences).to(device)
            #             print(x)
                        user_index = user_index.to(device)
                        # Mode "Teacher forcing"
                        x, y = x[:-1], x[1:]
#                         print(x.shape)
                        yhat, hidden = model.forward(x)
                        l = loss(yhat.view(-1,yhat.size(2)), y.view(-1))
                        l.backward()
                        optim.step()
                model.eval()
                with torch.no_grad():
                    for batch in valid_loader:
                        sequences, user_index = batch
                        x = preprocessor.process_batch(sequences).to(device)
                        user_index = user_index.to(device)
                        x, y = x[:-1], x[1:]
                        yhat, hidden = model(x)
                        evaluation = loss(yhat.view(-1,yhat.size(2)),y.view(-1))
                        cle=(hidden_dim, embed_d, learning_rate, epochs)
                        perf.setdefault(cle, evaluation)
                if best_eval is None or evaluation < best_eval:
                    best_eval = evaluation
                    best_params = {
                        'hidden_dim': hidden_dim,
                        'embedding_dim_values': embed_d,
                        'learning_rate': learning_rate,
                        'epochs': epochs
                    }

print("Meilleurs hyperparamètres : ", best_params)
print("Meilleure eval : ", best_eval)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Meilleurs hyperparamètres :  {'hidden_dim': 100, 'embedding_dim_values': 100, 'learning_rate': 0.001, 'epochs': 20}
Meilleure eval :  tensor(3.9270)


In [23]:
print("Meilleurs hyperparamètres : ", best_params)
print("Meilleure eval : ", best_eval)

Meilleurs hyperparamètres :  {'hidden_dim': 100, 'embedding_dim_values': 100, 'learning_rate': 0.001, 'epochs': 20}
Meilleure eval :  tensor(3.9270)


In [24]:
#Valeur de cross entropie associée à chaque combinaison d"hyperparamètres testés (sur l'ensemble de validation)
print(perf)

{(10, 10, 0.001, 10): tensor(4.4636), (10, 10, 0.001, 20): tensor(4.3150), (10, 10, 0.001, 30): tensor(4.2256), (10, 10, 0.001, 50): tensor(4.1538), (10, 10, 0.001, 100): tensor(4.0733), (10, 10, 0.001, 200): tensor(4.0405), (10, 10, 0.01, 10): tensor(4.4476), (10, 10, 0.01, 20): tensor(4.2911), (10, 10, 0.01, 30): tensor(4.2277), (10, 10, 0.01, 50): tensor(4.1355), (10, 10, 0.01, 100): tensor(4.0724), (10, 10, 0.01, 200): tensor(4.0065), (10, 50, 0.001, 10): tensor(4.3982), (10, 50, 0.001, 20): tensor(4.2229), (10, 50, 0.001, 30): tensor(4.1633), (10, 50, 0.001, 50): tensor(4.0750), (10, 50, 0.001, 100): tensor(4.0327), (10, 50, 0.001, 200): tensor(4.0044), (10, 50, 0.01, 10): tensor(4.3891), (10, 50, 0.01, 20): tensor(4.2376), (10, 50, 0.01, 30): tensor(4.1471), (10, 50, 0.01, 50): tensor(4.0924), (10, 50, 0.01, 100): tensor(4.0407), (10, 50, 0.01, 200): tensor(4.0126), (10, 100, 0.001, 10): tensor(4.4011), (10, 100, 0.001, 20): tensor(4.2061), (10, 100, 0.001, 30): tensor(4.1325), (